In [7]:
from numpy import exp, log, sqrt, empty, sum
import numpy as np
from scipy import integrate as ode, optimize as root

In [8]:
%run phys.ipynb

In [10]:
# state vector indices and units     
class indices:
    def __init__(self, si, n, fn):
        self.p, self.p_unit = 0, si.pascal
        self.T, self.T_unit = 1, si.kelvin
        self.q, self.q_unit = 2, si.dimensionless
        self.t_unit = si.second
        self.r_unit = si.metre
        self.n_unit = 1/si.kilogram
        
        assert(n > 0)
        self.x = slice(3, 3 + n)
        self.n = self.x.stop
        self.x_unit = fn.x_unit

In [12]:
class x_id:
    def __init__(self, si):
        self.x_unit = si.metre
    def x(self, r): return r
    def r(self, x): return x
    def dx_dr(self, r): return 1
    
class x_ln:
    def __init__(self, si):
        self.si = si
        self.r0 = 1 * si.metre
        self.x_unit = si.dimensionless
    def x(self, r): return log(r / self.r0)
    def r(self, x): return self.r0 * exp(x)
    def dx_dr(self, r): return 1/r
    
class x_p2:
    def __init__(self, si):
        self.x_unit = si.metre**2
    def x(self, r): return r**2
    def r(self, x): return sqrt(x)
    def dx_dr(self, r): return 2*r

In [14]:
# parcel model with monodisperse aerosol/droplet population
class eqsys:
    def __init__(self, ph, ix, fn):
        self.ix = ix
        self.fn = fn
        self.ph = ph
        
    def __call__(self, t, y):
        ix = self.ix
        fn = self.fn
        ph = self.ph
        
        t = t * ix.t_unit
        p = y[ix.p] * ix.p_unit
        q = y[ix.q] * ix.q_unit
        T = y[ix.T] * ix.T_unit
        x = y[ix.x] * ix.x_unit

        r = fn.r(x)
        dp_dt = ph.dp_dt(p, T, self.w(t),q)
        dr_dt = ph.dr_dt(r, T, ph.RH(T,p,q), self.kp, self.rd) 
        dq_dt = ph.dq_dt(self.nd, r, dr_dt)
        dT_dt = ph.dT_dt(T, p, dp_dt, q, dq_dt)
        
        dy_dt = empty(ix.n)
        dy_dt[ix.T] = (dT_dt / (ix.T_unit / ix.t_unit)).to(si.dimensionless)
        dy_dt[ix.p] = (dp_dt / (ix.p_unit / ix.t_unit)).to(si.dimensionless)
        dy_dt[ix.q] = (dq_dt / (ix.q_unit / ix.t_unit)).to(si.dimensionless)
        dy_dt[ix.x] = ((fn.dx_dr(r) * dr_dt) / (ix.x_unit / ix.t_unit)).to(si.dimensionless)
         
        return dy_dt

In [15]:
#wszystkie rd?
def parcel(*, si, t, T0, p0, w, q0, kp, rd, nd, dt_max):
    assert len(rd) == len(nd)
    nr = len(rd)

    fn = x_ln(si)
    ix = indices(si, nr, fn)
    ph = phys(si)
    
    sys = eqsys(ph, ix, fn)
    sys.w = w
    sys.kp = kp # TODO: muultiple kappas
    sys.rd = rd.to(ix.r_unit)
    sys.nd = nd.to(ix.n_unit)
        
    r0 = empty(nr) * ix.r_unit
    RH0 = ph.RH(T0, p0, q0)
    for i in range(nr):
        a = (sys.rd[i]/ix.r_unit).to(si.dimensionless)
        b = (ph.r_cr(kp, sys.rd[i], T0)/ix.r_unit).to(si.dimensionless) 
        f = lambda r: (
            ph.dr_dt(r * ix.r_unit, T0, RH0, kp, sys.rd[i]) / (ix.r_unit/ix.t_unit)
        ).to(si.dimensionless)        
        r0.magnitude[i] = root.brentq(f, a, b)
        
    x0 = fn.x(r0)
        
    y0 = empty(ix.n)
    y0[ix.T] = (T0/ix.T_unit).to(si.dimensionless)
    y0[ix.p] = (p0/ix.p_unit).to(si.dimensionless)
    y0[ix.q] = (q0/ix.q_unit).to(si.dimensionless)
    y0[ix.x] = (x0/ix.x_unit).to(si.dimensionless)
#    y0[ix.dr_dt] =  (phys.dr_dt(x * ix.x_unit, T0, RH0, kp, sys.rd[i]) /ix.x_unit/ix.t_unit).to(si.dimensionless)
        
    integ = ode.solve_ivp(sys, [0,t / ix.t_unit], y0, method='BDF', max_step=dt_max/ix.t_unit)
    assert integ.success, integ.message
    return integ, sys